# Designing networks

The combination of agents and locations that exist in a model makes the life of an agent-based modeler much easier.
Managing the relationships between agents in agent-based models is now very convenient.
However, it is still very tedious to connect a large number of agents through a specific network structure.
This is where the real magic of pop2net comes into play.

pop2net enables you to design a specific network structure by the definition of a special kind of location class: the `MagicLocation` class.
A `MagicLocation` can be considered as an *Interface* with a special syntax to create a certain number of location instances that connect specific agents in a specific way.
The assignment of agents to locations by the definition of a `MagicLocation` is primarely done through agent attributes.
Hence, **the creation of networks using pop2net makes the most sense when agents have informative attributes.**
However, even when the agents in your simulation do not have any attributes the `MagicLocation` class makes it easy to quickly connect agents in a certain way, for instance using classic network models.

The creation of the network based on what is defined in a `MagicLocation` is managed by the `Creator` class.
The `Creator` *reads* the definition of a `MagicLocation` and translates it into a certain number of certain location instances and then assigns the agents to the corresponding location instances.

## The Creator class

In order to *design* a specific network in pop2net, the first thing we need is an instance of the `Creator` class.
The purpose of the `Creator` is to generate agents and locations as well as to connect agents and locations.
Hence, the first step after creating a `Model` is the creation of an instance of `Creator`.

In [101]:
import pop2net as p2n

model = p2n.Model()
creator = p2n.Creator(model)

### Generating Agents

Although the main task of pop2net and the `Creator` class is connecting existing agents via the definition of `MagicLocation`s, the `Creator` can also generate agents for you.
A special feature of the `Creator` class is that it has some convenient methods to create agents based on micro-level data.
This makes it easy to create *empirical* agents from survey data, for instance.

Important to know is that while the generation of locations highly depents on the agents and their attributes, the generation of agents is independent from the generation of locations and their connection.
This may be different from classic network generators in which you generate nodes and edges in one step.

**However, you do not have to generate your agent population using pop2net or the `Creator` class. If you plan to create your agents in your own way and just want to connect your agents using pop2net and the `Creator` class, you can skip this subsection!**

The simplest way to generate agents using a `creator` is by using the method `creator.create_agents()` and define the desired number or agents by setting a certain `n`:

In [102]:
creator.create_agents(n=10)

AgentList (10 objects)

In [103]:
model.agents

AgentList (10 objects)

That is not very impressive.
We could have done the same using a simple for-loop.

Let's reset everything by removing those agents from the model:

In [104]:
model.remove_agents(model.agents)

In [105]:
model.agents

AgentList (0 objects)

Now let's have a look at an example for what pop2net was originally designed:
The creation of a population of agents based on survey data and their flexible connection via locations.

For the following example, we assume that we want to create a network model for schools and have access to a sample of individual data collected in a school.

This is our example data:

In [106]:
import pandas as pd

df_school = pd.read_csv("example_school_data.csv", sep=";")
df_school.head()

,status,gender,grade,hours,friend_group
0,pupil,m,1.0,4,1
1,pupil,w,1.0,4,3
2,pupil,m,1.0,4,2
3,pupil,w,1.0,4,2
4,pupil,m,2.0,4,11


In [107]:
df_school.tail()

,status,gender,grade,hours,friend_group
37,teacher,w,NaN,9,0
38,teacher,m,NaN,10,0
39,teacher,w,NaN,5,0
40,social_worker,w,NaN,6,0
41,social_worker,w,NaN,4,0


To create agents from this dataset, we can simply use the method `creator.create_agents()`.
This method creates one agent object for each row in the given dataset.
Each column is translated into an agent attribute with the corresponding value.

In [108]:
creator.create_agents(df=df_school)

AgentList (42 objects)

The created agents are added to the model's network and thus appear in `model.agents`:

In [109]:
model.agents

AgentList (42 objects)

Let's have look at the attributes of the first agent:

In [110]:
vars(model.agents[0])

{'_var_ignore': [],
 'id': 11,
 'type': 'Agent',
 'log': {},
 'model': Model,
 'p': {},
 'status': 'pupil',
 'gender': 'm',
 'grade': 1.0,
 'hours': 4,
 'friend_group': 1}

If we do not insert an agent class via the parameter `agent_class`, the default agent class of pop2net is used to create the agent instances, but we could also use our own agent class (that inherits from `p2n.Agent`):

In [111]:
class MyAgent(p2n.Agent):
    pass


model = p2n.Model()
creator = p2n.Creator(model)
agents = creator.create_agents(
    df=df_school,
    agent_class=MyAgent,
)
agents

AgentList (42 objects)

In [112]:
type(agents[0])

__main__.MyAgent

For now, this is enough information on how to create agents based on micro-level data using p2n.
More useful options for the generation of agents based on micro-level data can be found in chapter *From survey participants to agents*.

### The Magiclocation

To generate locations using the `Creator` class, we have to use it in combination with the `MagicLoation` class.
The `MagicLocation` can be considered as an interface to controll the automatic generation of locations and assignment of agents to the locations by the `creator`.

The `MagicLocation` class has various attributes and methods which can be customized by the user to specify which kind of locations are created and how the agents are assigned to which location instances.
The attributes and methods of the `MagicLocation` are *read* by the `creator` and then translated into the bipartite network of agents and locations.
Here is an overview of the attributes and methods of the `MagicLocation` class:

In [113]:
class MyLocation(p2n.MagicLocation):
    n_agents: int | None = None
    overcrowding: bool | None = None
    only_exact_n_agents: bool = False
    n_locations: int | None = None
    static_weight: bool = False
    recycle: bool = True

    def filter(self, agent: p2n.Agent) -> bool:
        """Assigns only agents with certain attributes to this location."""
        return True

    def split(self, agent: p2n.Agent) -> str | float | str | list | None:
        """Creates seperated location instances for each value of an agent-attribute."""
        return None

    def bridge(self, agent: p2n.Agent) -> float | str | list | None:
        """Create locations with one agent for each unique value returned."""
        return None

    def stick_together(self, agent: p2n.Agent) -> str | float:
        """Ensures that agents with a shared value on an attribute are assigned to the same
        location instance."""
        return agent.id

    def weight(self, agent: p2n.Agent) -> float:
        """Defines the edge weight between the agent and the location instance."""
        return 1

    def nest(self) -> p2n.Location | None:
        """Ensures that the agents assigned to the same instance of this location class
        are also assigned to the same instance of the returned location class."""
        return None

    def melt(self) -> list[p2n.Location] | tuple[p2n.Location]:
        """Merges the agents assigned to the instances of the returned location classes
        into one instance of this location class."""
        return []

    def project_weights(self, agent1: p2n.Agent, agent2: p2n.Agent) -> float:
        """Calculates the edge weight between two agents that are assigned to the same location
        instance."""
        return min([self.get_weight(agent1), self.get_weight(agent2)])

### Generating locations (and assigning agents)

We start by just using the default class of `MagicLocation` without defining any further attributes or methods.
Every location class defined by the user must inherit - directly or indirectly - from `p2n.MagicLocation`.
We call our first location `ClassRoom`.

In [114]:
class ClassRoom(p2n.MagicLocation):
    pass

Now we use the `creator.create_locations()` to create the location instances and - and this is important - to assign the agents to the location instances:

In [115]:
creator.create_locations(location_classes=[ClassRoom])

LocationList (1 object)

In [116]:
model.locations

LocationList (1 object)

`creator.create_locations()` has not only created the location instances with respect to the given agent population, but has also assigned the agents to the location according to the rules specified by the location classes.
We can check this by looking at the bipartite network:

In [117]:
inspector = p2n.NetworkInspector(model)
inspector.plot_bipartite_network()

The node in the center of the graph is the location.
All other nodes are agents.
This means that if we use the default location class without further customization, only one location instance is created to which all agents are connected.
Since each agent is assigned to this one location instance, the result is a fully connected agent graph:

In [118]:
inspector.plot_agent_network()

### Generate agents and locations in one step

Before diving into all the details of the definition of location classes, let's simplify the process of generating agents and locations.
The method `create()` combines `create_agents()` and `create_locations()` into one simple method.
However, note that `create()` always creates the agents based on a given dataset.
If you already have a population of agents, use `create_locations()` instead. 

In [119]:
model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

# Let the Creator create agents and locations
creator.create(
    df=df_school,
    location_classes=[ClassRoom],
)

(AgentList (42 objects), LocationList (1 object))

### Setting the location size

In the next step, we specify the number of people in one classroom.
In this example, we assume tiny classrooms of four agents.
To set the number of agents per location, we need to set the class attribute `n_agents` to the desired value.

In [120]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4

In [121]:
model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

(AgentList (42 objects), LocationList (10 objects))

In [122]:
inspector.plot_bipartite_network()

In [123]:
inspector.plot_agent_network()

The network diagrams above now show multiple clusters.
Each cluster represents one classroom.
If we set a specific size for a location, the Creator creates as many location instances with that size as needed.
The agents are then assigned randomly to one of these location instances.
As you can see, some classrooms have have more than four members because the number of agents assigned to classrooms cannot be divided exactly by the desired number of four.

The `overcrowding` attribute determines how the number of required locations is rounded.
By default, `overcrowding` is set to `None` which means that the number of required locations is either rounded up or rounded down using `round`.
Below we change `overcrowding` to `False` to create one more location instance to avoid *overcrowding* the classrooms.

In [124]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4
    overcrowding = False

In [125]:
model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

(AgentList (42 objects), LocationList (11 objects))

In [126]:
inspector.plot_bipartite_network()

The plot above now shows that there is now one class room more which has two members.
We could also set `overcrowding` to True to always round down the number of necessary location instances.

If we do not want locations that are either overcrowded or undercrowded but only locations that have exactly the size we defined, we could use the attribute `only_exact_n_agents` to `True`:

In [127]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4
    only_exact_n_agents = True


model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

inspector.plot_bipartite_network()

As a consequence, two agents are not assigned to any location.

### Defining the number of locations

The attribute `n_agents` implicitly changes the number of the created locations.
Using the attribute `n_locations`, you can also set the number of locations explicitly:

In [128]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4
    n_locations = 4


model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

inspector.plot_bipartite_network()

While in the example above four classrooms with the defined size of `n_agents = 4` are created, in the example below `n_agents` is not set explicitly:

In [129]:
class ClassRoom(p2n.MagicLocation):
    n_locations = 4


model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

inspector.plot_bipartite_network()

### Specifying location visitors

The classrooms above are made of all agents.
But in many cases we want specific locations to be exclusively accessible to certain agents.
For this scenario the method `filter()` exists.
If this method returns `True`, an agent gets assigned to an instance of this location class.
The most common way to use this method is to specify a condition that requires a certain agent attribute to contain a certain value.

In this example we want classrooms to be only accessible for pupils.

In [130]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

In [131]:
model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

(AgentList (42 objects), LocationList (8 objects))

In [132]:
inspector.plot_bipartite_network(agent_color="status")

Now classrooms consist only of pupils, while all other agents do not belong to any location.

By the way: Besides looking at the network graph, the function `inspector.location_information()` and `inspector.location_crosstab()` provide usefull overviews of the created location instances and the assigned agents:

In [133]:
inspector.location_information(
    select_locations=ClassRoom,
    agent_attributes=["grade", "status"],
    output_format="df",
)

,location_id,grade,status,location_type
0,0,1.0,pupil,ClassRoom
1,0,1.0,pupil,ClassRoom
2,0,1.0,pupil,ClassRoom
3,0,1.0,pupil,ClassRoom
4,1,2.0,pupil,ClassRoom
5,1,2.0,pupil,ClassRoom
6,1,2.0,pupil,ClassRoom
7,1,2.0,pupil,ClassRoom
8,2,3.0,pupil,ClassRoom
9,2,3.0,pupil,ClassRoom


In [134]:
inspector.location_crosstab(
    select_locations=ClassRoom,
    agent_attributes=["status"],
    output_format="df",
)

[index  location_id status  count location_type
 0                0  pupil      4     ClassRoom
 1                1  pupil      4     ClassRoom
 2                2  pupil      4     ClassRoom
 3                3  pupil      4     ClassRoom
 4                4  pupil      4     ClassRoom
 5                5  pupil      4     ClassRoom
 6                6  pupil      4     ClassRoom
 7                7  pupil      4     ClassRoom]

### Building separated locations

The above table shows that the classrooms are not separated by grade.
To seperate agents by grade, we could define one location class for each grade and use `filter()` to assign only agents with a specific grade value to a specific location.

A more convenient way to do it is to use the method `split()`.
For each agent, the method `split()` returns one value.
For each unique value, seperated location instances are created.
In this case, the method `split()` returns the attribute `grade` for each agents.
Thus, the Creator builds seperate classroom instances for each unique value of the agent attribute `grade`.

In [135]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

In [136]:
model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

(AgentList (42 objects), LocationList (8 objects))

In [137]:
inspector.plot_bipartite_network(agent_color="grade")

If we use `split()` in combination with `n_locations`, for each subgroup the desired number of locations gets realized:

In [138]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4
    n_locations = 1

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

In [139]:
model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

(AgentList (42 objects), LocationList (4 objects))

In [140]:
inspector.plot_bipartite_network(agent_attrs=df_school.columns, agent_color="grade")

### Keeping agents together

In the following plot the nodes are colored by their attribute `friend_group`.
It shows that the members of friend groups are distributed over different classrooms.

In [141]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

In [142]:
model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

(AgentList (42 objects), LocationList (8 objects))

In [143]:
inspector.plot_bipartite_network(agent_attrs=df_school.columns, agent_color="friend_group")

Although this is a very realistic situation, in this example, we want that all friend group members are always in the same class.
To implement that, we use the location method `stick_together()`:
For each agent, the method `stick_together()` returns a specific value.
Agents with the same return value are sticked together.

In [144]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

    def stick_together(self, agent):
        return agent.friend_group

In [145]:
model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

(AgentList (42 objects), LocationList (8 objects))

In [146]:
inspector.plot_bipartite_network(agent_attrs=df_school.columns, agent_color="friend_group")

### Edge weights

Until now, all edges between nodes have a weight of `1`.
The location method `weight()` can be used to set different weights.
In the following, we set the weight of all edges generated by a classroom to `5`.
This number could, for instance, represent that agents are together in classrooms for five hours.

In [147]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

    def stick_together(self, agent):
        return agent.friend_group

    def weight(self, agent):
        return 5

In [148]:
model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

agents, locations = creator.create(df=df_school, location_classes=[ClassRoom])

In [149]:
inspector.plot_bipartite_network(agent_attrs=df_school.columns, agent_color="grade")

In [150]:
inspector.plot_agent_network(agent_attrs=df_school.columns, agent_color="grade")

To implement individual weights between an agent and a location, we could also let `weight()` return an agent attribute. 
In this case we use the agent attribute `agent.hours`:

In [151]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

    def stick_together(self, agent):
        return agent.friend_group

    def weight(self, agent):
        return agent.hours


model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

(AgentList (42 objects), LocationList (8 objects))

In [152]:
inspector.plot_bipartite_network(agent_attrs=df_school.columns, agent_color="grade")

In [153]:
inspector.plot_agent_network(agent_attrs=df_school.columns, agent_color="grade")

As the value returned by `location.weight()` refers to the weight between the agent and the location, all the weights between the agents and the locations must be somehow combined when determining the weight between agents (aka graph projection).
The location method `project_weights()` defines how those weights are combined.
By default, `project_weights()` returns the smallest weight of the two to be combined.
The code cell below shows how `project_weights()` combines the two weights by default.
In this example, we keep this way of combining the weights, but this method could be easily rewritten.

In [154]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

    def stick_together(self, agent):
        return agent.friend_group

    def weight(self, agent):
        return agent.hours

    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

Note that we use the method `location.get_weight()` to access the weight between the agent and the location.

In [155]:
model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

inspector.plot_agent_network(agent_attrs=df_school.columns, agent_color="grade")

### Bringing together different agents

So far, we are able to connect agents who share a certain attribut value.
But what if we want to explicitly connect agents who have different values on a certain attribute?
One solution could be to simply give those agents we want to be in the same location the same value on a certain attribute and then define a location class that brings together these agents.



#### MagicLocation.bridge()

In [156]:
# a location for teachers and pupils
class ClassRoom(p2n.MagicLocation):
    def bridge(self, agent):
        return agent.status

In [157]:
model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

inspector.plot_agent_network(agent_attrs=df_school.columns, agent_color="status")

In [158]:
inspector.plot_bipartite_network(agent_attrs=df_school.columns, agent_color="status")

Another possibility is to *melt* different locations into one location.
To do this, we have to define at least three location classes:  Two or more locations that are the components that get melted into one location and one location that melts those components together.
Those locations that 

Assume we want to create classrooms that consist of one teacher and four pupils.
To create such a location, we first define a `MeltLocation` (`TeachersInClassRoom`) that consists of only one teacher.
Then we define a `MeltLocation` (`PupilsInClassRoom`) that consists of four pupils.
Finally, we define a location (`ClassRoom`) that uses the method `melt()` to melt the two previously defined locations into one location.
The method `melt()` must return a tuple or list of at least two location classes that shall be melted into one.

In [159]:
# a location for teachers
class TeachersInClassRoom(p2n.MeltLocation):
    n_agents = 1

    def filter(self, agent):
        return agent.status == "teacher"


# a location for pupils
class PupilsInClassRoom(p2n.MeltLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"


# a location for teachers and pupils
class ClassRoom(p2n.MagicLocation):
    def melt(self):
        return TeachersInClassRoom, PupilsInClassRoom

In [160]:
model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])

inspector.plot_bipartite_network(agent_attrs=df_school.columns, agent_color="status")

Now we bring back all other settings we made so far:

In [161]:
class TeachersInClassRoom(p2n.MeltLocation):
    n_agents = 1

    def filter(self, agent):
        return agent.status == "teacher"


class PupilsInClassRoom(p2n.MeltLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

    def stick_together(self, agent):
        return agent.friend_group


class ClassRoom(p2n.MagicLocation):
    def melt(self):
        return TeachersInClassRoom, PupilsInClassRoom

    def weight(self, agent):
        return agent.hours

    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])


model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])
inspector.plot_bipartite_network(agent_attrs=df_school.columns, agent_color="status")

In [162]:
class TeachersInClassRoom(p2n.MeltLocation):
    n_agents = 1

    def filter(self, agent):
        return agent.status == "teacher"


class PupilsInClassRoom(p2n.MeltLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

    def stick_together(self, agent):
        return agent.friend_group


class ClassRoom(p2n.MagicLocation):
    def melt(self):
        return TeachersInClassRoom, PupilsInClassRoom

    def weight(self, agent):
        return agent.hours

    def project_weights(self, agent1, agent2) -> float:
        statuses = [agent1.status, agent2.status]
        if "pupil" in statuses and "teacher" in statuses:
            return 1  # min([self.get_weight(agent1), self.get_weight(agent2)])
        else:
            return 0


model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom])
inspector.plot_agent_network(agent_attrs=df_school.columns, agent_color="status")

### More than one location

The melting of locations combines different locations into one location, but does not create multiple different location classes.
If we want to generate multiple different location types, we have to simply feed more than one location class into the `Creator`.

In the following, we introduce a `School` as a second type of location.
Each school has about `20` members.
(In order to keep the code clean, we skip the melting of locations temporarely.)

In [163]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

    def stick_together(self, agent):
        return agent.friend_group

    def weight(self, agent):
        return agent.hours

    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])


class School(p2n.MagicLocation):
    n_locations = 2


model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom, School])
inspector.plot_agent_network(agent_attrs=df_school.columns, agent_color="status")

In [164]:
inspector.plot_bipartite_network(location_color="type")

In [165]:
inspector.plot_agent_network(agent_attrs=df_school.columns, agent_color="status")

### Nesting locations
The plot above shows two big clusters.
Each of those clusters represents one school.
The plot above also shows something unrealistic: The schools are connected because members of one class are not always in the same school.
We can use the `School`-method `stick_together()` to fix this issue.
This works because whenever an agent is assigned to a location instance, the agent gets a new attribute named after the location class.
This new attribute is set to a location instance identifier value.

In [166]:
class ClassRoom(p2n.MagicLocation):
    n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

    def stick_together(self, agent):
        return agent.friend_group

    def weight(self, agent):
        return agent.hours

    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])


class School(p2n.MagicLocation):
    n_locations = 2

    def stick_together(self, agent):
        return agent.ClassRoom


model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom, School])

(AgentList (42 objects), LocationList (10 objects))

In [167]:
inspector.plot_bipartite_network(agent_color="status", location_color="type")

In [168]:
inspector.plot_agent_network(agent_attrs=df_school.columns, agent_color="status")

Note that it is very important that the agents get assigned to classrooms before getting assigned to schools.
This means that the order of the creation of the locations is important and, hence, the order of the location classes given to the `location_classes` argument.

If we build schools before classrooms, the above method does not work the way intended:

In [169]:
model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[School, ClassRoom])

inspector.plot_agent_network(agent_attrs=df_school.columns, agent_color="status")

In [170]:
inspector.plot_bipartite_network()

As we saw, the method `stick_together()` can be used to nest locations into other locations.
However, this approach is limited because we can only specify one location class as the return value of `stick_together()`.

Another way to nest locations into other locations is to use the location method `nest()`.
The method `nest()` can return `None` or a location class.
If `nest()` returns a location class, the location is nested into the returned location class.
For instance, to nest classrooms within schools, we must define the method `nest()` for the location `ClassRoom` and let this method return `School`.
Again, the order of location creation plays a crucial role: The level-1 location must always be created after the level-2 location.

In [171]:
class ClassRoom(p2n.MagicLocation):
    def setup(self):
        self.n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

    def stick_together(self, agent):
        return agent.friend_group

    def weight(self, agent):
        return agent.hours

    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

    def nest(self):
        return School


class School(p2n.MagicLocation):
    n_locations = 2


model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[School, ClassRoom])

inspector.plot_agent_network(agent_attrs=df_school.columns, agent_color="status")

In [172]:
inspector.plot_bipartite_network()

`nest()` allows us to nest as many locations in as many levels as we want. [Anwendungsbeispiel?]
However, `nest()` has one disadvantage: Because the agents are first grouped into the level-2 location and then into the level-1 location, specific compositions defined at level 1 are not considered when grouping the agents into the level-2 locations.

The following example demonstrates that:

In [173]:
class TeachersInClassRoom(p2n.MeltLocation):
    def setup(self):
        self.n_agents = 1

    def filter(self, agent):
        return agent.status == "teacher"


class PupilsInClassRoom(p2n.MeltLocation):
    def setup(self):
        self.n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade


class ClassRoom(p2n.MagicLocation):
    def melt(self):
        return [TeachersInClassRoom, PupilsInClassRoom]

    def weight(self, agent):
        return agent.hours * 10

    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])

    def nest(self):
        return School


class School(p2n.MagicLocation):
    n_locations = 2


model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[School, ClassRoom])
inspector.plot_bipartite_network()

In [174]:
inspector.location_crosstab(
    select_locations=[ClassRoom],
    agent_attributes="status",
    output_format="df",
)

[index  location_id   status  count location_type
 0                0    pupil      5     ClassRoom
 1                0  teacher      1     ClassRoom
 2                1    pupil      4     ClassRoom
 3                1  teacher      1     ClassRoom
 4                2    pupil      4     ClassRoom
 5                2  teacher      1     ClassRoom
 6                3    pupil      4     ClassRoom
 7                3  teacher      1     ClassRoom
 8                4    pupil      3     ClassRoom
 9                4  teacher      1     ClassRoom
 10               5    pupil      4     ClassRoom
 11               5  teacher      1     ClassRoom
 12               6    pupil      4     ClassRoom
 13               6  teacher      1     ClassRoom
 14               7    pupil      4     ClassRoom
 15               7  teacher      1     ClassRoom]

As you can see in the graph and the table above, the compositions defined by `ClassRoom` are not always met.
That is due to the fact that the agents are first assigned to the schools independently of the settings defined by `ClassRoom`.
When the classrooms are built, there are not always the *necessary* agents in each school needed to meet the composition defined in `ClassRoom`.
This might not always be a problem.
However, if we want to ensure that classrooms always have the defined composition of agents, they have to be created before the schools are created and, thus, we have to use the method `stick_together()` to nest classrooms into schools:

In [175]:
class TeachersInClassRoom(p2n.MeltLocation):
    def setup(self):
        self.n_agents = 1

    def filter(self, agent):
        return agent.status == "teacher"


class PupilsInClassRoom(p2n.MeltLocation):
    def setup(self):
        self.n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

    def stick_together(self, agent):
        return agent.friend_group


class ClassRoom(p2n.MagicLocation):
    def melt(self):
        return [TeachersInClassRoom, PupilsInClassRoom]

    def weight(self, agent):
        return agent.hours * 10

    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])


class School(p2n.MagicLocation):
    n_locations = 2

    def stick_together(self, agent):
        return agent.ClassRoom


model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(df=df_school, location_classes=[ClassRoom, School])
inspector.plot_agent_network(agent_attrs=df_school.columns, agent_color="status")

It is also possible to combine both nesting techniques.
We could first nest the class `ClassRoom` into `School` using `stick_together()` in order to get the composition wanted for `ClassRoom` and then use `nest()` to nest further locations into `School`, as it is done in the next example:

In [176]:
class TeachersInClassRoom(p2n.MeltLocation):
    def setup(self):
        self.n_agents = 1

    def filter(self, agent):
        return agent.status == "teacher"


class PupilsInClassRoom(p2n.MeltLocation):
    def setup(self):
        self.n_agents = 4

    def filter(self, agent):
        return agent.status == "pupil"

    def split(self, agent):
        return agent.grade

    def stick_together(self, agent):
        return agent.friend_group


class ClassRoom(p2n.MagicLocation):
    def melt(self):
        return [TeachersInClassRoom, PupilsInClassRoom]

    def weight(self, agent):
        return agent.hours * 10

    def project_weights(self, agent1, agent2) -> float:
        return min([self.get_weight(agent1), self.get_weight(agent2)])


class School(p2n.MagicLocation):
    n_locations = 2

    def stick_together(self, agent):
        return agent.ClassRoom


class SoccerTeam(p2n.MagicLocation):
    n_agents = 11

    def nest(self):
        return School


model = p2n.Model()
creator = p2n.Creator(model)
inspector = p2n.NetworkInspector(model)

creator.create(
    df=df_school,
    location_classes=[
        ClassRoom,  # nested into School using `stick_together()`
        School,
        SoccerTeam,  # nested into School using `nest()`
    ],
)
inspector.plot_agent_network(agent_attrs=df_school.columns, agent_color="status")

In [177]:
inspector.plot_bipartite_network()

This way we can nest multiple locations into one level-2 location and at the same time ensure that the composition of agents defined in `ClassRoom` is met in each school.
The order of the locations plays a crucial role:
1. The classrooms with the desired compositions are created
2. The schools are created keeping together whole classrooms
3. Soccer teams are created within the schools